## Preparación de Datos
1. Split de datos:
   - Train/Test/Validation
   - Mantener estratificación si es necesario
2. Procesamiento de variables (fit solo en train):
   - Codificación de variables categóricas
   - Normalización/Estandarización de variables numéricas
3. Balanceo de clases (solo en train si es necesario):
   - Oversampling
   - Undersampling
   - Técnicas híbridas

### Librerías

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import re

### Datos

In [3]:
df1 = pd.read_csv(r'C:\Users\nuria\OneDrive\Escritorio\ML_laptops\notebooks\DataFrame_laptops_limpio_1')
df1.head()

,Unnamed: 0,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,...,velocidad_cpu_ghz,marca_gpu,modelo_gpu,OpSys_general,2 in 1 Convertible,Gaming,Netbook,Notebook,Ultrabook,Workstation
0,0,1223,Dell,Inspiron 5567,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,3,256GB SSD,...,2.5,AMD,Radeon R7 M445,Windows,0,0,0,1,0,0
1,1,78,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,3,2TB HDD,...,2.5,Intel,HD Graphics 620,Sin OS,0,0,0,1,0,0
2,2,1267,Dell,XPS 13,2 in 1 Convertible,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i5 7Y54 1.2GHz,3,256GB SSD,...,1.2,Intel,HD Graphics 615,Windows,1,0,0,0,0,0
3,3,161,Dell,Inspiron 5579,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 8550U 1.8GHz,3,256GB SSD,...,1.8,Intel,UHD Graphics 620,Windows,1,0,0,0,0,0
4,4,922,LG,Gram 14Z970,Ultrabook,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,3,512GB SSD,...,2.7,Intel,HD Graphics 620,Windows,0,0,0,0,1,0


In [4]:
df1.isnull().sum()

Unnamed: 0            0
laptop_ID             0
Company               0
Product               0
TypeName              0
Inches                0
ScreenResolution      0
Cpu                   0
Ram                   0
Memory                0
Gpu                   0
OpSys                 0
Weight                0
Price_euros           0
Resolución            0
tipo_pantalla         0
memoria               0
tipo_memoria          0
tipo_cpu              0
Marca_cpu             0
Serie_cpu             0
Modelo_cpu            0
velocidad_cpu_ghz     0
marca_gpu             0
modelo_gpu            0
OpSys_general         0
2 in 1 Convertible    0
Gaming                0
Netbook               0
Notebook              0
Ultrabook             0
Workstation           0
dtype: int64

In [6]:
# Convertir a numérico usando el método de codificación
df1['OpSys'] = df1['OpSys'].astype('category').cat.codes
df1['velocidad_cpu_ghz'] = df1['velocidad_cpu_ghz'].astype('category').cat.codes

In [ ]:
# Convertir todas las columnas a numéricas, forzando a NaN donde no se pueda
# df_numeric = df1.apply(pd.to_numeric, errors='coerce')

## Procesado para ML

In [28]:
df1.head(7)

,Unnamed: 0,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,...,velocidad_cpu_ghz,marca_gpu,modelo_gpu,OpSys_general,2 in 1 Convertible,Gaming,Netbook,Notebook,Ultrabook,Workstation
0,0,1223,Dell,Inspiron 5567,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,3,256GB SSD,...,16,AMD,Radeon R7 M445,Windows,0,0,0,1,0,0
1,1,78,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,3,2TB HDD,...,16,Intel,HD Graphics 620,Sin OS,0,0,0,1,0,0
2,2,1267,Dell,XPS 13,2 in 1 Convertible,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i5 7Y54 1.2GHz,3,256GB SSD,...,3,Intel,HD Graphics 615,Windows,1,0,0,0,0,0
3,3,161,Dell,Inspiron 5579,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 8550U 1.8GHz,3,256GB SSD,...,8,Intel,UHD Graphics 620,Windows,1,0,0,0,0,0
4,4,922,LG,Gram 14Z970,Ultrabook,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,3,512GB SSD,...,18,Intel,HD Graphics 620,Windows,0,0,0,0,1,0
5,5,1040,HP,ProBook 650,Notebook,14.0,1366x768,Intel Core i5 7200U 2.5GHz,1,500GB HDD,...,16,Intel,HD Graphics 620,Windows,0,0,0,1,0,0
6,6,1039,HP,Elitebook 820,Ultrabook,12.5,1366x768,Intel Core i5 7200U 2.5GHz,1,256GB SSD,...,16,Intel,HD Graphics 620,Windows,0,0,0,0,1,0


### 1. Definir X e y

In [21]:
X = df1.drop(['laptop_ID','Company', 'Product', 'TypeName', 'ScreenResolution', 'Cpu', 'Memory', 'marca_gpu','modelo_gpu','OpSys_general', 'Gpu', 'Price_euros','Resolución','tipo_pantalla','memoria', 'Marca_cpu','Serie_cpu','Modelo_cpu', 'Weight',	'tipo_memoria',	'tipo_cpu'], axis=1)
y = df1['Price_euros'].copy()
X.shape

(912, 11)

In [18]:
y.shape

(912,)

### 2. Dividir X_train, X_test, y_train, y_test

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [23]:
X_train

,Unnamed: 0,Inches,Ram,OpSys,velocidad_cpu_ghz,2 in 1 Convertible,Gaming,Netbook,Notebook,Ultrabook,Workstation
25,25,14.0,1,5,18,0,0,0,1,0,0
84,84,15.6,1,2,11,0,0,0,1,0,0
10,10,15.6,3,5,16,0,0,0,1,0,0
342,342,15.6,1,4,2,0,0,0,1,0,0
890,890,13.3,3,5,16,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
106,106,15.6,3,5,7,0,0,0,1,0,0
270,270,15.6,3,5,16,0,0,0,1,0,0
860,860,14.0,3,5,16,0,0,0,0,1,0
435,435,15.6,3,4,11,0,0,0,1,0,0


### 3. Asignar el modelo (vacío) a una variable

In [24]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()

regressor.fit(X_train, y_train) # Entrenar

LinearRegression()

### Sacar métricas, valorar el modelo
RMSE para la competición

In [37]:
from sklearn.metrics import root_mean_squared_error

predictions = regressor.predict(X_test)

rmse = root_mean_squared_error(y_test, predictions)

In [27]:
print(rmse)

427.29591893538634


## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

### 2. Carga los datos de `test.csv` para predecir.

In [30]:
X_pred = pd.read_csv(r"C:\Users\nuria\OneDrive\Escritorio\ML_laptops\data\raw_data\test.csv")
X_pred.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,539,Asus,Zenbook UX510UW-FI095T,Notebook,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,8GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 960M,Windows 10,2kg
1,327,Asus,ZenBook UX410UA-GV183T,Notebook,14.0,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,2kg
2,563,Mediacom,SmartBook 130,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Atom x5-Z8350 1.44GHz,4GB,32GB Flash Storage,Intel HD Graphics,Windows 10,1.35kg
3,13,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.8GHz,16GB,256GB SSD,AMD Radeon Pro 555,macOS,1.83kg
4,935,HP,EliteBook 850,Ultrabook,15.6,Full HD 1920x1080,Intel Core i7 6500U 2.5GHz,8GB,256GB SSD,AMD Radeon R7 M365X,Windows 10,1.84kg


In [31]:
X_pred.shape

(391, 12)

In [38]:
from sklearn.metrics import root_mean_squared_error

predictions = regressor.predict(X_test)

rmse = root_mean_squared_error(y_test, predictions)

In [42]:
predictions_submit = regressor.predict(X_pred)
predictions_submit

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- 2 in 1 Convertible
- Gaming
- Netbook
- Notebook
- Ultrabook
- ...


In [43]:
regressor

LinearRegression()

In [41]:
print(X_pred.isnull().sum())

Inches    0
Ram       0
OpSys     0
dtype: int64


IMPORTANTE: APLICAR LO MISMO A ESTOS DATOS QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [34]:
X_pred = X_pred[['Inches', 'Ram', 'OpSys']]
X_pred

,Inches,Ram,OpSys
0,15.6,8GB,Windows 10
1,14.0,8GB,Windows 10
2,13.3,4GB,Windows 10
3,15.4,16GB,macOS
4,15.6,8GB,Windows 10
...,...,...,...
386,13.3,8GB,Windows 10
387,13.3,8GB,Windows 10
388,15.6,6GB,Windows 10
389,14.0,8GB,Windows 7


### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle.

**¿Qué es lo que subirás a Kaggle?**

In [44]:
predictions_submit = regressor.predict(X_pred)
predictions_submit

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- 2 in 1 Convertible
- Gaming
- Netbook
- Notebook
- Ultrabook
- ...
